<a href="https://colab.research.google.com/github/bmnapoleao/SLR-Automated_selection_of_studies/blob/main/notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary: <todo-describe>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from IPython.display import display
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn import metrics
import networkx as nx
from sklearn.decomposition import PCA
import seaborn as sns

In [2]:
# Lista de artigos duplicados (encontrados a partir de um script python)
DUPLICATED_LST = ['Leveraging organizational climate theory for understanding industry-academia collaboration', 'Meeting Industry-Academia Research Collaboration Challenges with Agile Methodologies', 'Characterizing industry-academia collaborations in software engineering: evidence from 101 projects', 'Protocol and Tools for Conducting Agile Software Engineering Research in an Industrial-Academic Setting: A Preliminary Study', 'Industry-Academia research collaboration in software engineering: The Certus model', 'Fostering Industry-Academia Collaboration in Software Engineering Using Action Research: A Case Study', 'A case study of industry--academia communication in a joint software engineering research project', 'Lessons Learned on Research Co-Creation: Making Industry-Academia Collaboration Work', 'Industry-Academia Collaborations in Software Engineering: An Empirical Analysis of Challenges, Patterns and Anti-Patterns in Research Projects', 'Knowledge Management in University-Software Industry Collaboration']

In [3]:
len(DUPLICATED_LST)

10

# Number of features used in Feature Selection

In [4]:
# k_fs = 100  
# k_fs = 500  
# k_fs = 750  # DONE
# k_fs = 1000 # DONE
k_fs = 1500 
k_sufix = 'k{}'.format(k_fs)

# Choose dataset to use and where to save results

In [5]:
# To use full dataset (2hrs to run feature selection)
#   Training Paths to read from
# file_path_included_training = '/content/drive/MyDrive/cslr/bibs/training-sets/training_included.bib'
# file_path_excluded_training = '/content/drive/MyDrive/cslr/bibs/training-sets/training_excluded.bib'
# #   Testing Paths to read from
# file_path_included_testing = '/content/drive/MyDrive/cslr/bibs/testing-sets/testing_included_formated.bib'
# file_path_excluded_testing = '/content/drive/MyDrive/cslr/bibs/testing-sets/testing_excluded_formated.bib'
# #   Using complete dataset (result's dir on drive)
# result_file_path = '/content/drive/MyDrive/cslr/results-dec/report_{}.csv'.format(k_sufix)

###########################################################################################################

# # # To use small sample from dataset
# #   Training Paths to read from
file_path_included_training = '/content/drive/MyDrive/cslr/bibs/small-samples/training-set-included.bib'
file_path_excluded_training = '/content/drive/MyDrive/cslr/bibs/small-samples/training-set-excluded.bib'
#   Testing Paths to read from
file_path_included_testing =  '/content/drive/MyDrive/cslr/bibs/small-samples/testing-set-included.bib'
file_path_excluded_testing =  '/content/drive/MyDrive/cslr/bibs/small-samples/testing-set-excluded.bib'
##   Using small samples / part of dataset (result's dir on drive)
k_fs = 100  # Use small number of features
k_sufix = 'k{}'.format(k_fs)
result_file_path = '/content/drive/MyDrive/cslr/results-dec/to-remove-test.csv'


# Selection of Studies

Marcelo Costalonga


## Bib Parser

######  # BibParser(write_files=False, project_folder=project_folder, only_titles=True),

In [6]:
!pip install bibtexparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 194 kB/s 
  Created wheel for bibtexparser: filename=bibtexparser-1.4.0-py3-none-any.whl size=42443 sha256=e316b931b02e6cd557cb2cbb368ca676fd763057761a3884ec3ea70ded3b89f7
  Stored in directory: /root/.cache/pip/wheels/6d/48/ea/211993480bbd28915707cff265dc40aa95db736838b6d014a8
Successfully built bibtexparser


### Imports

In [7]:
import codecs, bibtexparser

import keras_preprocessing.sequence
import numpy as np

from keras.preprocessing.text import Tokenizer

### Loading bibs from drive (BibTexParser)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
def read_bib(file_path, set_name: str, was_accepted: bool):
    texts_list = list()
    titles_list = list()
    duplicated_titles = dict()

    with codecs.open(file_path, 'r', encoding='utf-8') as bib_file:
        db = bibtexparser.load(bib_file)
        for bib_index, entry in enumerate(db.entries, start=0):
            category = 'selecionado' if was_accepted == True else 'removido'
            title = entry['title']
            abstract = entry['abstract']
            year = entry['year']
            content = u'%s\n%s' % (title, abstract)
            folder = file_path.split('/')[2].split('-')[0]

            if (title not in titles_list): 
                titles_list.append(title)
                content = content.split('\n')[0]
                texts_list.append({
                    'title': title,
                    'content': content,
                    'category': category,
                    'year': int(year)
                })
            else:
                if title in duplicated_titles:
                    duplicated_titles[title] += 1  
                else:
                     duplicated_titles[title] = 2

        bib_file.close()

        if (len(duplicated_titles) > 0):
            print("\nFound {} duplicated entries on set: {}".format(len(duplicated_titles), set_name))
            print("Each of the following entries were found Nx times (number the same title was found) and were ignored:")
            for title in duplicated_titles:
                print("\t({}x) - {}".format(duplicated_titles[title], title))

        return texts_list, titles_list

In [10]:
# Initialize lists
texts_list_included_training = []
texts_list_excluded_training = []
texts_list_included_testing = []
texts_list_excluded_testing = []

# Read bibs according to each set (training/testing - excluded/included)
texts_list_included_training, titles_included_training = read_bib(file_path_included_training, 'Training - Included', True)
texts_list_excluded_training, titles_excluded_training = read_bib(file_path_excluded_training, 'Training - Excluded', False)
texts_list_included_testing, titles_included_testing = read_bib(file_path_included_testing, 'Testing - Included', True)
texts_list_excluded_testing, titles_excluded_testing = read_bib(file_path_excluded_testing, 'Training - Excluded', False)

print("\nNumber of entries in Training Set Included:", len(texts_list_included_training))
print("Number of entries in Training Set Excluded:", len(texts_list_excluded_training))
print("Number of entries in Testing Set Included:", len(texts_list_included_testing))
print("Number of entries in Testing Set Excluded:", len(texts_list_excluded_testing))
print("Total number of entries before removing dupplicates:", len(texts_list_included_training) + len(texts_list_excluded_training) + 
      len(texts_list_included_testing) + len(texts_list_excluded_testing))



Found 1 duplicated entries on set: Testing - Included
Each of the following entries were found Nx times (number the same title was found) and were ignored:
	(2x) - Protocol and Tools for Conducting Agile Software Engineering Research in an Industrial-Academic Setting: A Preliminary Study

Number of entries in Training Set Included: 16
Number of entries in Training Set Excluded: 18
Number of entries in Testing Set Included: 35
Number of entries in Testing Set Excluded: 29
Total number of entries before removing dupplicates: 98


# Treating duplicates entries

In [11]:
from copy import deepcopy
import traceback

def remove_duplicates(text_lst, title_lst):
    cp_text_lst = deepcopy(text_lst)
    cp_title_lst = deepcopy(title_lst)

    print("Before:", len(cp_text_lst))

    for txt_obj in text_lst:
        # if txt_obj['content'] in DUPLICATED_LST:
        if txt_obj['title'] in DUPLICATED_LST:
            # print(txt_obj['title'])
            if txt_obj in cp_text_lst:
                cp_text_lst.remove(txt_obj)
                cp_title_lst.remove(txt_obj['title'])

    print("After:", len(cp_text_lst))
    return cp_text_lst, cp_title_lst


# texts_list_included_testing = remove_duplicates(texts_list_included_testing)
texts_list_excluded_testing, titles_excluded_testing = remove_duplicates(
    texts_list_excluded_testing, titles_excluded_testing)

Before: 29
After: 29


In [12]:
print("Number of entries in Training Set Included:", len(texts_list_included_training))
print("Number of entries in Training Set Excluded:", len(texts_list_excluded_training))
print("Number of entries in Testing Set Included:", len(texts_list_included_testing))
print("Number of entries in Testing Set Excluded:", len(texts_list_excluded_testing))
print("Total number of entries after removing dupplicates:", len(texts_list_included_training) + len(texts_list_excluded_training) + 
      len(texts_list_included_testing) + len(texts_list_excluded_testing))

Number of entries in Training Set Included: 16
Number of entries in Training Set Excluded: 18
Number of entries in Testing Set Included: 35
Number of entries in Testing Set Excluded: 29
Total number of entries after removing dupplicates: 98


In [13]:
def get_difference(l1, s1):
    lst_copy = deepcopy(l1)
    for i in s1:
        if i in lst_copy:
            lst_copy.remove(i)
    return lst_copy

def validate_texts(train_in, train_ex, test_in, test_ex):
    # Assert there are no duplicates
    duplicated_entries = list()
    try:
        train_titles = train_in + train_ex
        train_tuple = set(train_in + train_ex)
        assert len(train_titles) == len(train_tuple)
    except AssertionError:
        print("\nTraining Sets have duplicated entries!!")
        print("Len l1+l2={} vs Len set={}".format(len(train_titles), 
                                                  len(train_tuple)))
        duplicated_entries = get_difference(train_titles, train_tuple)
        print("The Following Entries appear both in Included and Excluded \
        Training sets:\n", duplicated_entries)

    try:
        test_titles = test_in + test_ex
        test_tuple = set(test_in + test_ex)
        assert len(test_titles) == len(test_tuple)
    except AssertionError:
        print("\nTesting Sets have duplicates!!")
        print("Len l1+l2={} vs Len set={}".format(len(test_titles), 
                                                  len(test_tuple)))    
        duplicated_entries = get_difference(test_titles, test_tuple)
        print("The Following Entries appear both in Included and Excluded Testing sets:\n")
        for i in duplicated_entries:
            print("\t - {}".format(i))

    try:
        all_titles = train_titles + test_titles
        all_tuple = set(train_titles + test_titles)
        assert len(all_titles) == len(all_tuple)
    except AssertionError:
        print("\nTraining set contains entries that are also in Testing set!!")
        print("Len l1+l2={} vs Len set={}".format(len(all_titles), 
                                                  len(all_tuple)))    
        print("The Following Entries appear both in Testing and Training sets: \
        \n", duplicated_entries)

    assert len(duplicated_entries) == 0, "Found errors"
    return duplicated_entries

# Validate Entries
duplicated_entries = validate_texts(titles_included_training, titles_excluded_training, 
               titles_included_testing, titles_excluded_testing)



In [14]:
for i in duplicated_entries:
    print(i)

In [15]:
# Split into two lists (training / testing)
texts_list_training = texts_list_included_training + texts_list_excluded_training
texts_list_training = sorted(texts_list_training, key=lambda d: d['year'])

texts_list_testing = texts_list_included_testing + texts_list_excluded_testing
texts_list_testing = sorted(texts_list_testing, key=lambda d: d['year'])

all_texts_list = texts_list_training + texts_list_testing
all_texts_list = sorted(all_texts_list, key=lambda d: d['year'])

print(len(all_texts_list))
# for text in all_texts:
#     print(text, len(text))

98


## Text Filtering (preprocessing)

######  # TextFilterComposite([ LemmatizerFilter(), StopWordsFilter() ]),

### Imports

In [16]:
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [17]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

### Text Filters

In [18]:
class TextFilterComposite:
    def __init__ (self, filters):
        self._filters = filters

    def _filter (self, tokens):
        result = tokens
        for f in self._filters:
            result = f.filter(result)
        return (' ').join(result)

class LemmatizerFilter:
    def __init__ (self):
        print('===== Configure the lemmatizer =====')
        self._lemmatizer = WordNetLemmatizer()

    def filter (self, tokens):
        tags = pos_tag(tokens)
        return [ self._lemmatizer.lemmatize(token[0], pos=token[1][0].lower())
                    if token[1][0].lower() in ('a', 'n', 'v', 'r')
                    else self._lemmatizer.lemmatize(token[0])
                    for token in tags ]

class StopWordsFilter:
    def __init__ (self):
        print('===== Configuring stop words removal =====')

    def filter (self, tokens):
        return [ word for word in tokens
                 if not word.lower() in stopwords.words('english') ]


In [19]:
# Tokenization 
filters = [LemmatizerFilter(), StopWordsFilter()]
textFilterObj = TextFilterComposite(filters)

def filter_text(text_list):
    result = []
    for text in text_list:
        tokens = word_tokenize(text['content'])
        
        # Converts list to filtered str
        #   E.g.: ['Research', 'in', 'computer', 'science', ':', 'an', 'empirical', 'study'] -> 'Research computer science : empirical study'
        filtered_text = textFilterObj._filter(tokens)

        result.append({
            'content': filtered_text.lower(),
            'category': text['category'],
            'year': text['year']
        })
    return result


===== Configure the lemmatizer =====
===== Configuring stop words removal =====


In [20]:
## 2) Calling filter method two times for each list (test | train)
filtered_texts_list_testing = filter_text(texts_list_testing)
filtered_texts_list_training = filter_text(texts_list_training)

In [22]:
print("Size of Test set:", len(filtered_texts_list_testing))
print("Size of Trainset:", len(filtered_texts_list_training))

Size of Test set: 64
Size of Trainset: 34


## Generate Dataset

######  # GenerateDataset(TfidfVectorizer(ngram_range=(1,3), use_idf=True)),

### Imports

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
class GenerateDataset:
    def __init__ (self, vectorizer=TfidfVectorizer()):
        self._vectorizer = vectorizer

datasetGenerator = GenerateDataset(TfidfVectorizer(ngram_range=(1,3), use_idf=True))

In [25]:
def generate_dataset(filtered_text_list):
    texts = [ text_data['content'] for text_data in filtered_text_list ]

    # Category: 1 == 'selecionado' | 0 == 'removido'
    categories = [ 1 if text_data['category'] == 'selecionado' else 0 for text_data in filtered_text_list ]
    years = [ text_data['year'] for text_data in filtered_text_list ]
    features = datasetGenerator._vectorizer.fit_transform(texts)

    dataset = {
        'texts': texts,
        'features': features,
        'categories': np.array(categories),
        'years': years
    }

    print(dataset['features'].shape)
    return dataset


In [26]:
## 2) Calling method two times for each list (test | train)
dataset_training = generate_dataset(filtered_texts_list_training)
dataset_testing = generate_dataset(filtered_texts_list_testing)

(34, 534)
(64, 1063)


In [27]:
print(dataset_testing['features'].max) 

<bound method _minmax_mixin.max of <64x1063 sparse matrix of type '<class 'numpy.float64'>'
	with 1480 stored elements in Compressed Sparse Row format>>


# Uses Feature Selection

In [28]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV, VarianceThreshold, SelectKBest, chi2

In [29]:
class USESFeatureSelection:
    def __init__ (self, k=-1):
        assert k > 0
        self._k = k
        self._affinity_score = []

    def _affinity (self, word_frequency_column, categories, category):
        ncw = 0
        nc = 0
        nw = 0
        for i in range(0, len(categories)):
            if categories[i] == category: nc += 1
            if word_frequency_column[i] > 0: nw += 1
            if categories[i] == category and word_frequency_column[i] > 0: ncw += 1
        return ncw / (nc + nw - ncw)


    def _score (self, features, categories):
        n_words = features.shape[1]
        self._affinity_score = [
            self._affinity(features[:,i], categories, 1) -
            self._affinity(features[:,i], categories, 0)
            for i in range(0, n_words) ]
        return (self._affinity_score, [])

    def execute(self, dt):
        print('===== Feature selection - USES =====')
        X = dt['features']
        y = dt['categories']
        fs = SelectKBest(self._score, k=self._k)
        dt['features'] = fs.fit_transform(X, y)
        print(dt['features'].shape)
        return dt


# Set number of K features at the begining of the file

In [30]:
# Initialize Feature Selection Class
featureSelection = USESFeatureSelection(k=k_fs)

In [31]:
# Feature Selection for training
dataset_training_kN_fs = featureSelection.execute(dataset_training)

===== Feature selection - USES =====
(34, 100)


In [32]:
# Feature Selection for testing
dataset_testing_kN_fs = featureSelection.execute(dataset_testing)

===== Feature selection - USES =====
(64, 100)


### K-Fold Split

In [33]:
import random
from keras import layers
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn import tree, metrics, svm, naive_bayes, ensemble, linear_model, neural_network
from sklearn.model_selection import cross_validate, TimeSeriesSplit, GridSearchCV, train_test_split

#### Define K-Fold split method

In [34]:
class YearsSplit:
    def __init__ (self, n_splits=4, years=[]):
        self._n_splits = n_splits
        self._years = years
        self._test_indexes = []
        current = max(years)
        for i in range(n_splits):
            test_index = years.index(current)
            if len(years[test_index:]) < 5:
                current = max(years[:test_index])
                test_index = years.index(current)

            self._test_indexes.append(test_index)
            current = max(years[:test_index])

    def split (self, X, y, groups=None):
        previous = len(self._years)
        for test_index in self._test_indexes:
            train = [ i for i in range(test_index) ]
            test = [ i for i in range(test_index, previous) ]
            previous = test_index
            yield train, test


### Define simple classifier (SVM and DT)

In [35]:
class SimpleClassifier:
    def __init__ (self, seed, n_splits=5):
        self._seed = seed
        self._n_splits = n_splits

    def execute (self, dataset_train, dataset_test):
        print("Executing...")
        X = dataset_train['features']
        y = dataset_train['categories']   
        
        groups = dataset_train['years']
        random.seed(self._seed)

        kfold = YearsSplit(n_splits=self._n_splits, years=groups)                 
        model = self.get_classifier(X, y)   # Pega modelo de classifier 

        scores = cross_validate(model, X, y, cv=kfold, scoring=['f1_macro', 'precision_macro', 'recall_macro'])
        print("OUR APPROACH F-measure: %s on average and %s SD" %
                (scores['test_f1_macro'].mean(), scores['test_f1_macro'].std()))
        print("OUR APPROACH Precision: %s on average and %s SD" %
                (scores['test_precision_macro'].mean(), scores['test_precision_macro'].std()))
        print("OUR APPROACH Recall: %s on average and %s SD" %
                (scores['test_recall_macro'].mean(), scores['test_recall_macro'].std()))

        X_train = dataset_train['features']
        y_train = dataset_train['categories']
        X_test = dataset_test['features']
        y_test = dataset_test['categories']

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test) 
        
        return y_pred

#### Extend Classifiers 

In [36]:
class DecisionTreeClassifier (SimpleClassifier):
    def __init__ (self, seed=42, criterion='entropy', n_splits=5):
        SimpleClassifier.__init__(self, seed, n_splits)
        self.classifier_name = 'decision_tree'
        self._criterion = criterion

    def get_classifier (self, X, y):
        print('===== Decision Tree Classifier =====')
        print('===== Hyperparameter tunning  =====')
        model = tree.DecisionTreeClassifier()
        params = {
            'criterion': ["gini", "entropy"],
            'max_depth': [10, 50, 100, None],
            'min_samples_split': [2, 10, 100],
            'class_weight': [None, 'balanced']
        }
        cfl = GridSearchCV(model, params, cv=5, scoring='recall')
        cfl.fit(X, y)
        for param, value in cfl.best_params_.items():
            print("%s : %s" % (param, value))

        model = tree.DecisionTreeClassifier(random_state=self._seed)
        model.set_params(**cfl.best_params_)
        return model


class SVMClassifier (SimpleClassifier):
    def __init__ (self, seed, n_splits=5):
        SimpleClassifier.__init__(self, seed, n_splits=n_splits)
        self.classifier_name = 'svm'

    def get_classifier (self, X, y):
        print('===== SVM Classifier =====')
        print('===== Hyperparameter tunning  =====')
        params = {
            #'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
            'kernel': ['linear', 'rbf'],
            'C': [1, 10, 100],
            #'degree': [1, 2, 3],
            #'coef0': [0, 10, 100],
            'tol': [0.001, 0.1, 1],
            'class_weight': ['balanced', None]
        }
        model = svm.SVC(random_state=self._seed, probability=True)
        cfl = GridSearchCV(model, params, cv=5, scoring='accuracy')
        cfl.fit(X, y)
        for param, value in cfl.best_params_.items():
            print("%s : %s" % (param, value))

        model = svm.SVC(random_state=self._seed, probability=True)
        model.set_params(**cfl.best_params_)
        return model


### Define fixed seed (42) and number of splits to initialize Classifiers

In [37]:
n_splits = 3

dt_classifier = DecisionTreeClassifier(seed=42, criterion='gini', n_splits=n_splits)
svm_classifier = SVMClassifier(42, n_splits=n_splits)

## Execute classifier for specific kN features


## Execute and store results in 'dataset_scores' dict

In [38]:
dataset_scores = dict()

#### DT classifier

In [39]:
# Execute DT classifier
dt_y_pred_kN = dt_classifier.execute(dataset_training_kN_fs, dataset_testing_kN_fs)

Executing...
===== Decision Tree Classifier =====
===== Hyperparameter tunning  =====
class_weight : None
criterion : gini
max_depth : None
min_samples_split : 10
OUR APPROACH F-measure: 0.35000000000000003 on average and 0.08923543557893895 SD
OUR APPROACH Precision: 0.3888888888888889 on average and 0.23516718079411197 SD
OUR APPROACH Recall: 0.5 on average and 0.08164965809277258 SD


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### SVM classifier

In [40]:
# Execute SVM classifier
svm_y_pred_kN = svm_classifier.execute(dataset_training_kN_fs, dataset_testing_kN_fs)

Executing...
===== SVM Classifier =====
===== Hyperparameter tunning  =====
C : 1
class_weight : balanced
kernel : rbf
tol : 0.001
OUR APPROACH F-measure: 0.9576719576719577 on average and 0.05986089152901989 SD
OUR APPROACH Precision: 0.9583333333333334 on average and 0.05892556509887896 SD
OUR APPROACH Recall: 0.9666666666666667 on average and 0.04714045207910316 SD


### Compare results:

In [41]:
# DataFrame antes do FeatureSelection
df_training1 = pd.DataFrame.from_dict(filtered_texts_list_training)
df_testing1 = pd.DataFrame.from_dict(filtered_texts_list_testing)

df_training1.head(10) # Exemplo do estado do dataframe

,content,category,year
0,evolution information technology : implication...,removido,1989
1,partnership software industry academia,removido,2003
2,guide project management body knowledge ( pmbo...,removido,2004
3,design science information systems research,removido,2004
4,model technology transfer practice,selecionado,2006
5,sustainable technology transfer,selecionado,2006
6,action research path solid research culture is...,selecionado,2007
7,continuing professional development practition...,removido,2007
8,factors influencing industrial practices softw...,removido,2007
9,3-step knowledge transition : case study archi...,selecionado,2008


In [42]:
# DataFrame após resultado do FeatureSelection
df_training2 = pd.DataFrame.from_dict(dataset_training_kN_fs)
df_testing2 = pd.DataFrame.from_dict(dataset_testing_kN_fs)

df_training2.head(10) # Exemplo do estado do dataframe

,texts,features,categories,years
0,evolution information technology : implication...,"(0, 72)\t0.21273963586201802",0,1989
1,partnership software industry academia,"(0, 0)\t0.2916151356420139\n (0, 14)\t0.223...",0,2003
2,guide project management body knowledge ( pmbo...,,0,2004
3,design science information systems research,"(0, 18)\t0.1549163429457114",0,2004
4,model technology transfer practice,"(0, 17)\t0.3030104455684067\n (0, 72)\t0.26...",1,2006
5,sustainable technology transfer,"(0, 67)\t0.45367693930992475\n (0, 68)\t0.4...",1,2006
6,action research path solid research culture is...,"(0, 2)\t0.15159151874347532\n (0, 3)\t0.163...",1,2007
7,continuing professional development practition...,,0,2007
8,factors influencing industrial practices softw...,"(0, 4)\t0.15607134211286572\n (0, 11)\t0.15...",0,2007
9,3-step knowledge transition : case study archi...,"(0, 4)\t0.17976818502746583\n (0, 5)\t0.206...",1,2008


In [43]:
# DataFrames para analisar DT e SVM após execução 
from copy import deepcopy

# DataFrame DT
# dt_df_training = pd.DataFrame.from_dict(dataset_training_kN_fs)
dt_df_testing = deepcopy(df_testing2)
dt_df_testing['predicted'] = dt_y_pred_kN

# DataFrame SVM
# svm_df_training = pd.DataFrame.from_dict(dataset_training_kN_fs)
svm_df_testing = deepcopy(df_testing2)
svm_df_testing['predicted'] = svm_y_pred_kN


In [44]:
df_testing = deepcopy(df_testing2)
df_testing['DT_pred'] = dt_y_pred_kN
df_testing['SVM_pred'] = svm_y_pred_kN


#### Decision Tree vs SVM

In [45]:
df_testing.head(20)

,texts,features,categories,years,DT_pred,SVM_pred
0,planning unknown : lessons learned ten months ...,"(0, 21)\t0.14958509341845988\n (0, 24)\t0.0...",1,2015,1,1
1,exploratory study technology transfer software...,"(0, 17)\t0.14232047367077214\n (0, 21)\t0.2...",1,2015,0,1
2,fast feedback cycles empirical software engine...,"(0, 13)\t0.19531677287385588\n (0, 14)\t0.2...",1,2015,1,1
3,integration se research industry : reflections...,"(0, 24)\t0.0934486830681684\n (0, 35)\t0.09...",1,2015,0,1
4,researcher ’ experiences supporting industrial...,"(0, 23)\t0.166421288531331\n (0, 38)\t0.088...",1,2016,0,0
5,softcoder approach : promoting software engine...,"(0, 0)\t0.0932000917621854\n (0, 17)\t0.085...",1,2016,0,0
6,research framework build spi proposal small or...,"(0, 35)\t0.09027327981061076",1,2016,0,0
7,academic industrial software testing conferenc...,"(0, 23)\t0.17069480186168143\n (0, 38)\t0.0...",0,2016,0,0
8,applying data analytics towards optimized issu...,"(0, 23)\t0.1362476225050109",0,2016,0,0
9,meeting industry-academia research collaborati...,"(0, 0)\t0.12545450748786866\n (0, 4)\t0.229...",1,2017,0,1


In [46]:
# 'SVM_pred' or 'DT_pred'
def get_real_negatives(classifier_type: str):
    return df_testing.loc[(df_testing['categories'] == 0) & (df_testing['categories'] == df_testing[classifier_type]), ['texts', 'categories', classifier_type]]

def get_real_positives(classifier_type: str):
    return df_testing.loc[(df_testing['categories'] == 1) & (df_testing['categories'] == df_testing[classifier_type]), ['texts', 'categories', classifier_type]]

def get_false_negative(classifier_type: str):
    return df_testing.loc[(df_testing['categories'] == 1) & (df_testing[classifier_type] == 0), ['texts', 'categories', classifier_type]]

def get_false_positive(classifier_type: str):
    return df_testing.loc[(df_testing['categories'] == 0) & (df_testing[classifier_type] == 1), ['texts', 'categories', classifier_type]] 

#### DT analysis

In [47]:
cls_type = 'DT_pred'
df_dt_real_negatives = get_real_negatives(cls_type)
df_dt_real_positives = get_real_positives(cls_type)
df_dt_false_negative = get_false_negative(cls_type)
df_dt_false_positive = get_false_positive(cls_type)

In [48]:
print("RN",len(df_dt_real_negatives))
print("RP", len(df_dt_real_positives))
print("FN", len(df_dt_false_negative))
print("FP", len(df_dt_false_positive))

RN 29
RP 5
FN 30
FP 0


In [49]:
df_dt_real_positives

,texts,categories,DT_pred
0,planning unknown : lessons learned ten months ...,1,1
2,fast feedback cycles empirical software engine...,1,1
22,"recruitment , engagement feedback empirical so...",1,1
23,continuous collaborative technology transfer :...,1,1
39,design science paradigm frame empirical softwa...,1,1


#### SVM analysis

In [50]:
cls_type = 'SVM_pred'
df_svm_real_negatives = get_real_negatives(cls_type)
df_svm_real_positives = get_real_positives(cls_type)
df_svm_false_negative = get_false_negative(cls_type)
df_svm_false_positive = get_false_positive(cls_type)

In [51]:
print("RN",len(df_svm_real_negatives))
print("RP",len(df_svm_real_positives))
print("FN",len(df_svm_false_negative))
print("FP",len(df_svm_false_positive))

print("\n",df_svm_false_positive)

RN 28
RP 26
FN 9
FP 1

                                                 texts  categories  SVM_pred
52  guiding selection research methodology industr...           0         1


In [52]:
df_svm_real_positives

,texts,categories,SVM_pred
0,planning unknown : lessons learned ten months ...,1,1
1,exploratory study technology transfer software...,1,1
2,fast feedback cycles empirical software engine...,1,1
3,integration se research industry : reflections...,1,1
9,meeting industry-academia research collaborati...,1,1
11,industry -- academia collaboration software te...,1,1
20,leveraging organizational climate theory under...,1,1
21,protocol tools conducting agile software engin...,1,1
22,"recruitment , engagement feedback empirical so...",1,1
23,continuous collaborative technology transfer :...,1,1


In [53]:
print(df_svm_real_positives)


                                                texts  categories  SVM_pred
0   planning unknown : lessons learned ten months ...           1         1
1   exploratory study technology transfer software...           1         1
2   fast feedback cycles empirical software engine...           1         1
3   integration se research industry : reflections...           1         1
9   meeting industry-academia research collaborati...           1         1
11  industry -- academia collaboration software te...           1         1
20  leveraging organizational climate theory under...           1         1
21  protocol tools conducting agile software engin...           1         1
22  recruitment , engagement feedback empirical so...           1         1
23  continuous collaborative technology transfer :...           1         1
24  towards model transfer knowledge software engi...           1         1
27  characterizing industry-academia collaboration...           1         1
29  contextu

## Convert dataframe to CSV and send to drive

In [54]:
df_report = df_testing

unused_columns = ['features', 'years']
df_report.drop(unused_columns, inplace=True, axis=1)
df_report.rename(columns = {'categories':'Was Selected?'}, inplace = True)

with codecs.open(result_file_path, 'w', encoding='utf-8') as report_file:
    df_report.to_csv(report_file, index=False)

# END